## DESNOYER Jeremy & PIQUET Lucas

## Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe spécifique qui générera un mot de passe en prennant **4 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [3]:
import numpy as np

from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre) : le générateur génère des mots de passe à partir des occurences des monogrammes
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [2]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values

In [8]:
freq_mono

array([0.1776, 0.0823, 0.0768, 0.0761, 0.073 , 0.0723, 0.0681, 0.0605,
       0.0589, 0.0534, 0.036 , 0.0332, 0.0324, 0.0272, 0.0134, 0.0127,
       0.011 , 0.0106, 0.008 , 0.0064, 0.0054, 0.0021, 0.0019, 0.0007,
       0.    , 0.    ])

Q: Quelles sont les 5 lettres les plus représentées ?

In [6]:
monogramme.sort_values(by='frequency', ascending=False)[:5]

,letters,frequency
0,E,0.1776
1,S,0.0823
2,A,0.0768
3,N,0.0761
4,T,0.0730


Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien *gérer* le cas ou la probabilité est nulle).

In [18]:
def entropie(freq):
    ent = 0
    for frequence in freq[freq!=0]:
        ent -= frequence*np.log2(frequence)
    return ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [24]:
print(f'{8*entropie(freq_mono)}')

31.676242429778338


En considérant que les 8 lettres du mdp sont indépendantes et issues de la loi X définie par les fréquences alors l'entropie du mdp et la somme des 8 entropies

Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [35]:
t = time.time()

mdp = np.random.choice(letters_mono, size=(8,100000), p= freq_mono)
t_100000 = time.time() - t
print(t_100000, 's')


0.0449981689453125 s


Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [34]:
lower_bound = 2**(entropie(freq_mono)*8)/4+1
print(f'Le minorant de G dans notre cas est {np.round(lower_bound,2)} essais')

Le minorant de G dans notre cas est 857904864.68 essais


Q: combien de temps cela prendra-t-il pour trouver un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [43]:
print(f'D\'après l\'entropie du devin il faut {np.round(t_100000*lower_bound/100000/60,2)} min')


D'après l'entropie du devin il faut 6.43 min


On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [44]:

bigramme = read_csv('bigramme.csv',keep_default_na=False)


#print(bigramme.head(5))
freq_bi = (bigramme['frequency']).values

letters_bi = (bigramme['letters']).values



Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [45]:
bigramme.sort_values(by='frequency', ascending=False)[:5]

,letters,frequency
122,ES,0.023809
117,EN,0.021248
82,DE,0.019570
290,LE,0.018845
357,NT,0.017009


Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [48]:
entropie(freq_bi)*4

30.142264046464188

Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?

Car l'univers de la variable alètoire défini par la répartition des bigrammes est de taille 26x26 donc l'information transmises par un couple de lettre est plus faible.
On a la propriété théorique suivante : $ H(X,Y) <= H(X) + H(Y)$ 

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [57]:
t = time.time()


mdp = np.random.choice(letters_bi, size=(4,100000), p= freq_bi)
t2_100000 = time.time() - t

print(t2_100000, 's')


0.044960975646972656 s


Q: calculer le minorant de $G$ pour ce modèle

In [52]:
lower_bound = 2**(entropie(freq_bi)*4)/4+1
print(f'Le minorant de G dans le cas bigramme est {np.round(lower_bound)} essais')

Le minorant de G dans le cas bigramme est 296254957.0 essais


Q: combien de temps cela prendra-t-il pour trouver un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [53]:
print(f'D\'après l\'entropie du devin il faut {np.round(t2_100000*lower_bound/100000/60,2)} min')


D'après l'entropie du devin il faut 2.03 min


Q: **Modèle Uniforme:** si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [61]:
freq_uniform = np.full(26, 1/26)
print(f'L\'entropie est de {entropie(freq_uniform)*8} par indépendance du tirage')

L'entropie est de 37.60351774512872 par indépendance du tirage


Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [59]:
t = time.time()


mdp = np.random.choice(letters_mono, size=(8,100000), p= freq_uniform)
t3_100000 = time.time() - t

print(t3_100000, 's')

0.05948305130004883 s


Q: calculer le minorant de $G$ pour ce modèle

In [63]:
lower_bound = 2**(entropie(freq_uniform)*8)/4+1
print(f'Le minorant de G dans le cas uniforme est {np.round(lower_bound)} essais')

Le minorant de G dans le cas uniforme est 52206766145.0 essais


Q: dans ce cas précis, quelle est la valeur exacte de $G$?

Dans le cas uniforme on a $26^8$ possibilités donc en moyenne on doit effectuer $26^8/2$ essais soit G = 104413532288

Q: combien de temps cela prendra-t-il pour trouver un mot de passe en utilisant le générateur codé précédemment ? (en minutes)

In [64]:
print(f'D\'après l\'entropie du devin il faut {np.round(t3_100000*lower_bound/100000/60,2)} min')


D'après l'entropie du devin il faut 517.57 min


Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur: (la personne qui génère le mot de passe)** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [66]:
## Fonction générant un mot de passe
def get_passwd():
    text_hugo = open("texteFrancais.txt","r")
    str_hugo = str(text_hugo.read())

    # On remplace des lettres avec accent avec des lettres sans accent
    str_hugo = str_hugo.replace("Â", "A")
    str_hugo = str_hugo.replace("Ù", "U")
    str_hugo = str_hugo.replace("Ô", "O")
    size_txt = len(str_hugo)

    idx_rand = np.random.randint(size_txt-4)
    #print(idx_rand)

    psswd = str_hugo[idx_rand:idx_rand+4]
    return(psswd)

In [69]:
get_passwd()

'EASO'

In [129]:
# Génération du dictionnaire
tab_passwd = []

frequence = []
# On exclue les couple de frequence null
#bigramme_pos = bigramme[bigramme['frequency']>0].reset_index()

for i in range(len(bigramme)):
    for j in range(len(bigramme)):
        tab_passwd.append(bigramme.loc[i,'letters'] + bigramme.loc[j,'letters'])
        frequence.append(bigramme.loc[i,'frequency'] *  bigramme.loc[j,'frequency'])

df_dict = pd.DataFrame({'mdp': tab_passwd,
                        'frequency': frequence})

In [135]:
df_dict.sort_values(by='frequency', ascending= False, inplace=True)
df_dict

,mdp,frequency
82594,ESES,0.000567
82589,ESEN,0.000506
79214,ENES,0.000506
55554,DEES,0.000466
82554,ESDE,0.000466
...,...,...
445059,ZIJR,0.000000
445060,ZIJS,0.000000
160666,JDRM,0.000000
160665,JDRL,0.000000


In [136]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
vec_nb_trials = np.zeros(nb_trial)
tab_passwd = df_dict['mdp'].values

for j in range(nb_trial):
    password = get_passwd()
    for i in range(tab_passwd.shape[0]):
        if tab_passwd[i] == password:
            vec_nb_trials[j] = i
            break 

In [137]:
print(f'En moyenne on a recquis {int(vec_nb_trials.mean())} essais pour cracker les mdp')

En moyenne on a recquis 13979 essais pour cracker les mdp


In [140]:
lower_bound = 2**(entropie(freq_bi)*2)/4+1
print(f'Le minorant de G dans le cas bigramme est {int(lower_bound)} essais')

Le minorant de G dans le cas bigramme est 8607 essais


Pas d'anomalie en est bien en présence d'une borne inférieur
Par la suite nous allons tester une deuxième méthode qui intégre également la probabilité d'observer les lettres 2 et 3 à la suite.

In [141]:
# Génération du dictionnaire
tab_passwd_2 = []

frequence_2 = []

for i in range(len(bigramme)):
    a = bigramme.loc[i,'letters']
    for j in range(len(bigramme)):
        b = bigramme.loc[j,'letters']
        c = a[1]+b[0]
        tab_passwd_2.append(a + b)
        frequence_2.append(bigramme.loc[i,'frequency'] *  bigramme.loc[j,'frequency'] * bigramme[bigramme.letters == c]['frequency'].values[0] )

df_dict_2 = pd.DataFrame({'mdp': tab_passwd_2,
                        'frequency': frequence_2})

In [142]:
df_dict_2.sort_values(by='frequency', ascending= False, inplace=True)
df_dict_2

,mdp,frequency
55789,DENT,0.000007
196397,LENT,0.000007
82594,ESES,0.000007
301853,RENT,0.000006
82589,ESEN,0.000006
...,...,...
267368,PFNK,0.000000
267369,PFNL,0.000000
267370,PFNM,0.000000
267371,PFNN,0.000000


In [145]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
vec_nb_trials_2 = np.zeros(nb_trial)
tab_passwd_2 = df_dict_2['mdp'].values

for j in range(nb_trial):
    password = get_passwd()
    for i in range(tab_passwd_2.shape[0]):
        if tab_passwd_2[i] == password:
            vec_nb_trials_2[j] = i
            break 

In [146]:
print(f'En moyenne on a recquis {int(vec_nb_trials_2.mean())} essais pour cracker les mdp')

En moyenne on a recquis 11664 essais pour cracker les mdp


On observe qu'avec cette 2ème méthode on a réussi à avoir une densité de probabilité plus efficace et réduire le nombre d'essais moyen requis

In [152]:
lower_bound = 2**(entropie(freq_uniform)*4)/4+1
print(f'Le minorant de G dans le cas uniforme est {np.round(lower_bound)} essais')

Le minorant de G dans le cas uniforme est 114245.0 essais


Si on utilise un générateur uniforme alors $G = 26^4/2= 228488$

Donc en utilisant la méthode bigramme on réduit drastiquement le nombre d'essais moyen à effectuer pour générer le bon mot de passe

## Conclusions 

- Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 
- Définior des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?

- Pour un défenseur les bonne pratiques serait de prendres des combinaisons de lettres peu représentées dans la langue française pour contrer ce type de méthode qui joue sur l'usage de combinaison de lettre plus fréquente pour réduire le nombre de recherche recquis pour craquer un mot de passe. Ou alors utiliser un générateur aléatoire qui suivent une loi uniforme pour maximiser l'entropie.
- Pour un attaquant il peux être avantageux d'apprendre au mieux la distribution des bigrammes, trigrammes, ect  pour minimiser le nombre d'essais recquis

## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/
